# Config for database

In [ ]:
from Utils import postgres_tool as postgres
import configparser
import os

config = configparser.ConfigParser()
config.read('./config/config.ini')
db_host = config['database_sv']['DB_SV_HOSTS']
db_port = config['database_sv'].getint('DB_SV_PORT')
db_username = config['database_sv']['DB_SV_USER']
db_password = config['database_sv']['DB_SV_PASSWORD']
db_database = config['database_sv']['DB_SV_DATABASE']
print(db_host )

In [ ]:
# import yaml

# database = yaml.load(open('./config/config.yml', 'r'), Loader=yaml.FullLoader)['database_sv']
# database

# conectDB

In [ ]:

conn = postgres.PostgresTool(db_host, db_username, db_port, db_password, db_database)

In [ ]:
conn.get_all_table()

In [ ]:
import pandas as pd 

data = pd.read_csv('./data/prediction.csv')

for i in range(len(data)):
    print(data.iloc[i].to_dict())
    conn.push_data('prediction', data.iloc[i].to_dict())


In [ ]:
conn.close()

# Use os file csv

In [ ]:
import numpy as np 
import pandas as pd 

import os
for dirname, _, filenames in os.walk('./db'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Insert data for database

In [ ]:
# conn.insert_data('./db\sport.csv')


In [ ]:
# conn.insert_data('./db/team.csv')


In [ ]:
# conn.insert_data('./db/date_id.csv')


In [ ]:
# conn.insert_data('./db/season.csv')

In [ ]:
# conn.insert_data('./db/date_id.csv')

In [ ]:
# conn.insert_data('./db/match.csv')


In [ ]:
# conn.insert_data('./db/prediction.csv')


In [ ]:
from datetime import datetime
import pytz
def get_us_date(timezone_name='US/Eastern'):
    us_timezone = pytz.timezone(timezone_name)
    utc_now = datetime.now(pytz.utc)
    us_now = utc_now.astimezone(us_timezone)
    return us_now.strftime('%Y-%m-%d')
CURRENT_DATE_US = get_us_date()
sport_id = "mlb"
query = f"SELECT idx,   id FROM date_id WHERE date_play = '{CURRENT_DATE_US}' AND sport_id = '{sport_id}' AND season = '2024'"
idx = conn.query(query, False)[0][0]
id_idx = conn.query(query, False)[0][1]
print(idx)
print(id_idx)

In [ ]:
querys1 = """SELECT CASE
    WHEN MAX(id) = COUNT(id) THEN 'Ascending'
    ELSE 'Not Ascending'
END AS result
FROM (
    SELECT id, ROW_NUMBER() OVER (ORDER BY id) AS rn
    FROM "match" 
) subquery
WHERE id <> rn;
"""

conn.query(querys1, False)[0][0]

In [ ]:
querys2 = """SELECT
    a.attname AS column_name
FROM
    pg_index i
    JOIN pg_attribute a ON a.attnum = ANY(i.indkey)
WHERE
    i.indrelid = 'match'::regclass
    AND i.indisprimary;
"""
conn.query(querys2, False)

In [ ]:
conn.close()

# END

In [ ]:
import redis
import psycopg2
from psycopg2 import sql
import json

# Cấu hình
REDIS_HOST = 'localhost'
REDIS_PORT = 6379
REDIS_DB = 0

# Kết nối đến Redis
redis_client = redis.StrictRedis(host=REDIS_HOST, port=REDIS_PORT, db=REDIS_DB)



def cache_insert_data(data):
    # Tạo một khóa duy nhất cho dữ liệu
    data_key = f"data:{data['id']}"
    
    # Lưu trữ dữ liệu vào Redis
    redis_client.set(data_key, json.dumps(data))
    
    # Chèn dữ liệu vào PostgreSQL
    # if insert_data(data):
    #     # Nếu chèn thành công, xóa dữ liệu được lưu trữ trong Redis
    #     redis_client.delete(data_key)
    # else:
    #     print("Chèn dữ liệu vào cơ sở dữ liệu thất bại. Dữ liệu được lưu trữ trong Redis vẫn còn.")

# Dữ liệu ví dụ
data_list = [
    {'id': 8670, 'column2': 1, 'column3': -1.5, 'column4': 1.5, 'column5': 8, 'column6': 313, 'column7': 4, 'column8': 1, 'column9': 4, 'team1': 'PIT', 'team2': 'CIN', 'status': 'Final', 'date': '2024-06-17', 'sport': 'mlb'},
    {'id': 8671, 'column2': 2, 'column3': -1.5, 'column4': 1.5, 'column5': 9.5, 'column6': 313, 'column7': 9, 'column8': 2, 'column9': 9, 'team1': 'PHI', 'team2': 'SD', 'status': 'Final', 'date': '2024-06-17', 'sport': 'mlb'},
    {'id': 8672, 'column2': 7, 'column3': 1.5, 'column4': -1.5, 'column5': 7, 'column6': 313, 'column7': 6, 'column8': 7, 'column9': 6, 'team1': 'MIA', 'team2': 'STL', 'status': 'Final', 'date': '2024-06-17', 'sport': 'mlb'},
    {'id': 8673, 'column2': 7, 'column3': -1.5, 'column4': 1.5, 'column5': 7.5, 'column6': 313, 'column7': 3, 'column8': 7, 'column9': 3, 'team1': 'TOR', 'team2': 'BOS', 'status': 'Final', 'date': '2024-06-17', 'sport': 'mlb'},
    {'id': 8674, 'column2': 1, 'column3': -1.5, 'column4': 1.5, 'column5': 8, 'column6': 313, 'column7': 2, 'column8': 1, 'column9': 2, 'team1': 'ATL', 'team2': 'DET', 'status': 'Final', 'date': '2024-06-17', 'sport': 'mlb'},
    {'id': 8675, 'column2': 14, 'column3': -1.5, 'column4': 1.5, 'column5': 8, 'column6': 313, 'column7': 2, 'column8': 14, 'column9': 2, 'team1': 'TEX', 'team2': 'NYM', 'status': 'Final', 'date': '2024-06-18', 'sport': 'mlb'},
    {'id': 8676, 'column2': 7, 'column3': -1.5, 'column4': 1.5, 'column5': 10.5, 'column6': 313, 'column7': 6, 'column8': 7, 'column9': 6, 'team1': 'CHC', 'team2': 'SF', 'status': 'Final', 'date': '2024-06-18', 'sport': 'mlb'},
    {'id': 8677, 'column2': 9, 'column3': 1.5, 'column4': -1.5, 'column5': 12, 'column6': 313, 'column7': 5, 'column8': 9, 'column9': 5, 'team1': 'COL', 'team2': 'LAD', 'status': 'Final', 'date': '2024-06-18', 'sport': 'mlb'},
    {'id': 8678, 'column2': 3, 'column3': 1.5, 'column4': -1.5, 'column5': 8.5, 'column6': 313, 'column7': None, 'column8': None, 'column9': 5, 'team1': 'LAA', 'team2': 'MIL', 'status': 'InProgress', 'date': '2024-06-18', 'sport': 'mlb'}
]

# Chèn dữ liệu
for data in data_list:
    cache_insert_data(data)
redis_client.get('data:8670')

In [ ]:
# ✅ Data inserted successfully into (8682, None, 1.5, -1.5, 8, 314, None, None, None, 'MIA', 'STL', 'Scheduled', '2024-06-18', 'mlb')! ✅
# ✅ Data inserted successfully into (8679, None, 1.5, -1.5, 8.5, 314, None, None, None, 'PIT', 'CIN', 'Scheduled', '2024-06-18', 'mlb')! ✅
# ✅ Data inserted successfully into (8680, None, -1.5, 1.5, 8.5, 314, None, None, None, 'PHI', 'SD', 'Scheduled', '2024-06-18', 'mlb')! ✅
# ✅ Data inserted successfully into (8681, None, 1.5, -1.5, 8.5, 314, None, None, None, 'CLE', 'SEA', 'Scheduled', '2024-06-18', 'mlb')! ✅
# ✅ Data inserted successfully into (8683, None, 1.5, -1.5, 8.5, 314, None, None, None, 'WSH', 'ARI', 'Scheduled', '2024-06-18', 'mlb')! ✅
# ✅ Data inserted successfully into (8684, None, -1.5, 1.5, 9, 314, None, None, None, 'NYY', 'BAL', 'Scheduled', '2024-06-18', 'mlb')! ✅
# ✅ Data inserted successfully into (8685, None, 1.5, -1.5, 8, 314, None, None, None, 'TOR', 'BOS', 'Scheduled', '2024-06-18', 'mlb')! ✅
# ✅ Data inserted successfully into (8686, None, -1.5, 1.5, 8.5, 314, None, None, None, 'ATL', 'DET', 'Scheduled', '2024-06-18', 'mlb')! ✅
# ✅ Data inserted successfully into (8687, None, -1.5, 1.5, 8, 314, None, None, None, 'MIN', 'TB', 'Scheduled', '2024-06-18', 'mlb')! ✅
# ✅ Data inserted successfully into (8688, None, 1.5, -1.5, 8, 314, None, None, None, 'TEX', 'NYM', 'Scheduled', '2024-06-19', 'mlb')! ✅
# ✅ Data inserted successfully into (8689, None, 1.5, -1.5, 9.5, 314, None, None, None, 'CHC', 'SF', 'Scheduled', '2024-06-19', 'mlb')! ✅
# ✅ Data inserted successfully into (8690, None, 1.5, -1.5, 8.5, 314, None, None, None, 'CHW', 'HOU', 'Scheduled', '2024-06-19', 'mlb')! ✅
# ✅ Data inserted successfully into (8691, None, 1.5, -1.5, 11.5, 314, None, None, None, 'COL', 'LAD', 'Scheduled', '2024-06-19', 'mlb')! ✅
# ✅ Data inserted successfully into (8692, None, 1.5, -1.5, 8.5, 314, None, None, None, 'LAA', 'MIL', 'Scheduled', '2024-06-19', 'mlb')! ✅
# ✅ Data inserted successfully into (8693, None, 1.5, -1.5, 8, 314, None, None, None, 'OAK', 'KC', 'Scheduled', '2024-06-19', 'mlb')! ✅


In [ ]:
from datetime import datetime
import pytz
def get_us_date(timezone_name='US/Eastern'):
    us_timezone = pytz.timezone(timezone_name)
    utc_now = datetime.now(pytz.utc)
    us_now = utc_now.astimezone(us_timezone)
    return us_now.strftime('%Y-%m-%d') 
print(get_us_date())
from datetime import datetime, timedelta
import pytz

def get_us_date_yesterday(timezone_name='US/Eastern'):
    us_timezone = pytz.timezone(timezone_name)
    utc_now = datetime.now(pytz.utc)
    us_now = utc_now.astimezone(us_timezone)
    us_yesterday = us_now - timedelta(days=1)
    return us_yesterday.strftime('%Y-%m-%d')

print(get_us_date_yesterday())


In [ ]:
YEAR_DATE_US = get_us_date_yesterday()
sport_id = "mlb"
season = "2024"
query = f"SELECT idx,   id FROM date_id WHERE date_play = '{YEAR_DATE_US}' AND sport_id = '{sport_id}' AND season = '{season}'"
idx = conn.query(query, False)[0][0]
id_idx = conn.query(query, False)[0][1]
print(idx)
print(id_idx)




In [ ]:
id_idx =333

In [ ]:
querys1 = f"""SELECT COUNT(*) FROM "match" m WHERE m.date_id = '{id_idx}' AND m.status != 'Final'"""
count = conn.query(querys1, False)[0][0]

print(count)


In [ ]:
import requests

url = 'https://api.nflpickwatch.com/v1/general/games/2024/116/mlb/REGULAR'

response = requests.get(url)

print(response.json)